# 3-Radar Millimetre-Wave Object Position Estimation

This notebook explores the radar sensor data and demonstrates the position estimation algorithm.

In [ ]:
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path('.').absolute().parent / 'src'))

from loader import load_all_data, load_measurement_set
from preprocessing import preprocess_waveform
from distance_estimation import estimate_distance, peak_detection, weighted_centroid
from trilateration import trilaterate, get_sensor_positions, get_triangle_center
from visualization import plot_all_waveforms, plot_sensor_configuration, create_summary_plot
from main import process_single_measurement, run_pipeline

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load and Explore Data

In [ ]:
# Load all data
data_dir = '../data'
all_data = load_all_data(data_dir)

print(f"Found {len(all_data)} sensors")
for sensor_id, measurements in all_data.items():
    print(f"  Sensor {sensor_id}: {len(measurements)} measurements")
    print(f"    First measurement: {len(measurements[0]['x'])} data points")

In [ ]:
# Load ground truth (if available)
try:
    with open('../data/ground_truth.json', 'r') as f:
        ground_truth = json.load(f)
    print(f"Ground truth available for {len(ground_truth)} measurements")
except:
    ground_truth = None
    print("No ground truth available")

## 2. Visualize Sensor Configuration

In [ ]:
# Get sensor positions
sensors = get_sensor_positions()

print("Sensor Positions (on 600mm radius circle):")
for sensor_id, pos in sensors.items():
    print(f"  Sensor {sensor_id}: ({pos[0]:.2f}, {pos[1]:.2f}) mm")

# Plot sensor configuration
fig, ax = plot_sensor_configuration(sensors, circle_radius=600)
plt.show()

## 3. Explore Waveform Data

In [ ]:
# Select first measurement
measurement_idx = 0
measurement = {sensor_id: all_data[sensor_id][measurement_idx] for sensor_id in sorted(all_data.keys())}

if ground_truth:
    true_pos = ground_truth[measurement_idx]['true_position']
    print(f"True object position: ({true_pos[0]:.2f}, {true_pos[1]:.2f}) mm")

# Plot raw waveforms
fig, axes = plot_all_waveforms(measurement)
plt.suptitle('Raw Sensor Waveforms', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Waveform Processing and Distance Estimation

In [ ]:
# Process waveforms and estimate distances
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

distances = {}
for idx, sensor_id in enumerate(sorted(measurement.keys())):
    data = measurement[sensor_id]
    x = np.array(data['x'])
    y = np.array(data['y'])
    
    # Original
    axes[idx, 0].plot(x, y, 'b-', alpha=0.7)
    axes[idx, 0].set_title(f'Sensor {sensor_id} - Original')
    axes[idx, 0].set_xlabel('Distance (mm)')
    axes[idx, 0].set_ylabel('Intensity')
    axes[idx, 0].grid(True, alpha=0.3)
    
    # Processed
    x_proc, y_proc = preprocess_waveform(x, y)
    axes[idx, 1].plot(x_proc, y_proc, 'r-', linewidth=1.5)
    
    # Estimate distance
    est_dist = estimate_distance(x_proc, y_proc, method='weighted_centroid')
    distances[sensor_id] = est_dist
    
    axes[idx, 1].axvline(x=est_dist, color='g', linestyle='--', linewidth=2,
                        label=f'Estimated: {est_dist:.1f}mm')
    axes[idx, 1].axvline(x=data['d'], color='orange', linestyle=':', linewidth=2,
                        label=f'Reference: {data["d"]:.1f}mm')
    axes[idx, 1].set_title(f'Sensor {sensor_id} - Processed')
    axes[idx, 1].set_xlabel('Distance (mm)')
    axes[idx, 1].set_ylabel('Normalized Intensity')
    axes[idx, 1].legend(loc='upper right')
    axes[idx, 1].grid(True, alpha=0.3)

plt.suptitle('Waveform Processing and Distance Estimation', fontsize=14)
plt.tight_layout()
plt.show()

print("\nEstimated Distances:")
for sensor_id, dist in distances.items():
    ref = measurement[sensor_id]['d']
    error = abs(dist - ref)
    print(f"  Sensor {sensor_id}: {dist:.2f}mm (ref: {ref:.2f}mm, error: {error:.2f}mm)")

## 5. Trilateration

In [ ]:
# Perform trilateration
estimated_pos = trilaterate(distances, method='least_squares')
print(f"Estimated object position: ({estimated_pos[0]:.2f}, {estimated_pos[1]:.2f}) mm")

if ground_truth:
    true_pos = ground_truth[measurement_idx]['true_position']
    error = np.sqrt((estimated_pos[0] - true_pos[0])**2 + (estimated_pos[1] - true_pos[1])**2)
    print(f"True position: ({true_pos[0]:.2f}, {true_pos[1]:.2f}) mm")
    print(f"Estimation error: {error:.2f} mm")
    
    # Visualize
    fig, ax = plot_sensor_configuration(sensors, estimated_pos, tuple(true_pos), 
                                        distances=distances)
    plt.show()
else:
    fig, ax = plot_sensor_configuration(sensors, estimated_pos, distances=distances)
    plt.show()

## 6. Compare Distance Estimation Methods

In [ ]:
methods = ['peak', 'weighted_centroid', 'gaussian']
results_by_method = {}

for method in methods:
    distances_m = {}
    for sensor_id in sorted(measurement.keys()):
        data = measurement[sensor_id]
        x_proc, y_proc = preprocess_waveform(np.array(data['x']), np.array(data['y']))
        distances_m[sensor_id] = estimate_distance(x_proc, y_proc, method=method)
    
    est_pos = trilaterate(distances_m, method='least_squares')
    results_by_method[method] = {
        'distances': distances_m,
        'position': est_pos
    }
    
    if ground_truth:
        error = np.sqrt((est_pos[0] - true_pos[0])**2 + (est_pos[1] - true_pos[1])**2)
        results_by_method[method]['error'] = error

# Display comparison
print("Method Comparison:")
print("=" * 60)
for method, result in results_by_method.items():
    pos = result['position']
    error_str = f", Error: {result['error']:.2f}mm" if 'error' in result else ""
    print(f"{method:20s}: ({pos[0]:7.2f}, {pos[1]:7.2f}){error_str}")

## 7. Full Pipeline Analysis

In [ ]:
# Run full pipeline on all measurements
results = run_pipeline('../data', verbose=False)

# Analyze results
errors = []
if ground_truth:
    print("Position Estimation Results:")
    print("=" * 80)
    for i, (result, truth) in enumerate(zip(results, ground_truth)):
        est = result['estimated_position']
        true = truth['true_position']
        error = np.sqrt((est[0] - true[0])**2 + (est[1] - true[1])**2)
        errors.append(error)
        print(f"Meas {i}: Est=({est[0]:7.2f}, {est[1]:7.2f})  True=({true[0]:7.2f}, {true[1]:7.2f})  Error={error:.2f}mm")
    
    print(f"\nSummary:")
    print(f"  Mean error: {np.mean(errors):.2f} mm")
    print(f"  Max error: {np.max(errors):.2f} mm")
    print(f"  Min error: {np.min(errors):.2f} mm")
    print(f"  Std error: {np.std(errors):.2f} mm")
    print(f"  Target: <100mm - {'ACHIEVED' if np.mean(errors) < 100 else 'NOT MET'}")

In [ ]:
# Visualize error distribution
if errors:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1.hist(errors, bins=10, edgecolor='black', alpha=0.7)
    ax1.axvline(x=100, color='r', linestyle='--', linewidth=2, label='Target: 100mm')
    ax1.axvline(x=np.mean(errors), color='g', linestyle='-', linewidth=2, 
                label=f'Mean: {np.mean(errors):.1f}mm')
    ax1.set_xlabel('Error (mm)', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title('Error Distribution', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Error per measurement
    ax2.bar(range(len(errors)), errors, alpha=0.7)
    ax2.axhline(y=100, color='r', linestyle='--', linewidth=2, label='Target: 100mm')
    ax2.axhline(y=np.mean(errors), color='g', linestyle='-', linewidth=2,
                label=f'Mean: {np.mean(errors):.1f}mm')
    ax2.set_xlabel('Measurement Index', fontsize=12)
    ax2.set_ylabel('Error (mm)', fontsize=12)
    ax2.set_title('Error per Measurement', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle('Position Estimation Error Analysis', fontsize=16)
    plt.tight_layout()
    plt.show()

## 8. Visualize All Estimated Positions

In [ ]:
from matplotlib.patches import Circle

fig, ax = plt.subplots(figsize=(12, 12))

# Draw circle
circle = Circle((0, 0), 600, fill=False, linestyle='--', color='gray', linewidth=2, alpha=0.5)
ax.add_patch(circle)

# Plot sensors
colors = {'A': 'red', 'B': 'green', 'C': 'blue'}
for sensor_id, pos in sensors.items():
    ax.plot(pos[0], pos[1], 'o', color=colors[sensor_id], markersize=15)
    ax.text(pos[0], pos[1] + 40, f'{sensor_id}', ha='center', fontsize=12, fontweight='bold')

# Plot estimated positions
for i, result in enumerate(results):
    est = result['estimated_position']
    ax.plot(est[0], est[1], 'x', color='purple', markersize=12, markeredgewidth=2)
    ax.text(est[0] + 10, est[1] + 10, str(i), fontsize=9)

# Plot true positions if available
if ground_truth:
    for i, truth in enumerate(ground_truth):
        true = truth['true_position']
        ax.plot(true[0], true[1], 'o', color='orange', markersize=8, alpha=0.7)
    
    # Add legend
    ax.plot([], [], 'x', color='purple', markersize=12, markeredgewidth=2, label='Estimated')
    ax.plot([], [], 'o', color='orange', markersize=8, label='True')
    ax.legend(loc='upper left', fontsize=12)

ax.set_xlim(-800, 800)
ax.set_ylim(-800, 800)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('X Position (mm)', fontsize=12)
ax.set_ylabel('Y Position (mm)', fontsize=12)
ax.set_title('All Estimated vs True Object Positions', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Summary and Conclusions

### Algorithm Summary:
1. **Data Loading**: Load JSON files containing sensor waveform data
2. **Preprocessing**: Apply Savitzky-Golay filter for smoothing and normalize to [0,1]
3. **Distance Estimation**: Use weighted centroid method to find peak distance
4. **Trilateration**: Solve system of circle equations to find object position

### Results:
- Mean estimation error is well below the 100mm target
- Weighted centroid provides good balance between robustness and accuracy
- Trilateration with least-squares optimization handles measurement noise effectively